In [2]:
import torch

import pyro
import pyro.distributions as dist

In [3]:
loc = 0.   # mean zero
scale = 1. # unit variance
normal = dist.Normal(loc, scale) # create a normal distribution object
x = normal.sample() # draw a sample from N(0,1)
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)

sample tensor(-0.3212)
log prob tensor(-0.9705)


In [4]:
#One of the core language primitives in Pyro is the pyro.sample statement. Using pyro.sample is as simple as calling a primitive stochastic function with one important difference:
x = pyro.sample("my_sample", dist.Normal(loc, scale))
print(x)

tensor(0.0179)


In [16]:
#A simple stochastic function:   
def weather():
    cloudy = pyro.sample('cloudy', dist.Bernoulli(0.3))
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = pyro.sample('temp', dist.Normal(mean_temp, scale_temp))
    return cloudy, temp.item()

for _ in range(3):
    print(weather())

('sunny', 85.33436584472656)
('cloudy', 47.766143798828125)
('sunny', 82.75843048095703)


In [12]:
cloudy = pyro.sample('cloudy', dist.Bernoulli(0.3))
cloudy

tensor(0.)

In [14]:
cloudy.item()

0.0

In [10]:
cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
cloudy

'sunny'

Because Pyro is embedded in Python, stochastic functions can contain arbitrarily complex deterministic Python and randomness can freely affect control flow. For example, we can construct recursive functions that terminate their recursion nondeterministically, provided we take care to pass pyro.sample unique sample names whenever it’s called. For example we can define a geometric distribution like so:

In [21]:
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), dist.Bernoulli(p))
    if x.item() == 0:
        return x
    else:
        return x + geometric(p, t + 1)

print(geometric(0.5))
print(geometric(0.7))
print(geometric(0.5))
print(geometric(0.3))

tensor(2.)
tensor(1.)
tensor(1.)
tensor(2.)


In [25]:
dist.Bernoulli(0.3)
#pyro.sample("x_{}".format(0), dist.Bernoulli(0.3))

Bernoulli()

The fact that Pyro supports arbitrary Python code like this—iteration, recursion, higher-order functions, etc.—in conjuction with random control flow means that Pyro stochastic functions are universal, i.e. they can be used to represent any computable probability distribution. As we will see in subsequent tutorials, this is incredibly powerful.

In [26]:
#just testing the python3 formatting
"asds{}".format(2)

'asds2'

In [30]:
#We are also free to define stochastic functions that accept as input or produce as output other stochastic functions:
def normal_product(loc, scale):
    z1 = pyro.sample("z1", dist.Normal(loc, scale))
    z2 = pyro.sample("z2", dist.Normal(loc, scale))
    y = z1 * z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", dist.Normal(0, 1))
    fn = lambda scale: normal_product(mu_latent, scale)
    return fn

print(make_normal_normal()(1.))

tensor(0.2036)


In [ ]:
#inference in Pyro